In [ ]:
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from pydantic import BaseModel

**NOTE**: please run this notebook from `./src`

In [ ]:
%cd ../..
%pwd

In [ ]:
assert Path.cwd().parts[-2:] == ("HyUi", "src")

In [ ]:
from mock import mock  # hence the need to get the directory set up

In [ ]:
vitals_file = mock.path_to_hdf_file("perrt")
df = pd.read_hdf(vitals_file)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
engine = mock.make_mock_db_in_memory("perrt")

In [ ]:
df.info()

In [ ]:
df["id_in_application"].value_counts()

In [ ]:
assert len(df["id_in_application"].value_counts()) == 9

## Data wrangling notes

In [ ]:
import api.perrt.wrangle as wng
from api.perrt.wrangle import wrangle

vitals_file = mock.path_to_hdf_file("perrt")
df = pd.read_hdf(vitals_file)

In [ ]:
df.head()

NB assumes repeating MRN but this is not true in the synth data
`df.mrn.value_counts()` # this will not help since you forced mrn to be unique in the sdv spec
hence the hack below to force some repeats in mrn

In [ ]:
import os

if os.getenv("ENV", "dev") == "dev":
    new_mrn = df.loc[:, "mrn"].sample(frac=1, replace=True).reset_index(drop=True)
    df["mrn"] = new_mrn

In [ ]:
df = wrangle(df)

In [ ]:
df.head()

In [ ]:
list(df.columns)